# YOLO - Inferencia
## Detección de objetos

**Autor original:** @theAIGuysCode<br>
**Adaptado:** @maxiyommi<br>
**Fecha:** jun 2021<br>
**Descripción:** Notebook para inferir resultados.

## Instalación de OpenCV

``` bash
    sudo apt install python3-opencv
    sudo apt-get install libopencv-dev
```

## Instalar zip

``` bash
    sudo apt install zip unzip
``` 

## Preparación del entorno

* Crear entorno virtual (solo la primera vez) dentro de la carpeta `/inference`.

``` bash
    python3 -m venv yolo_inference --system-site-packages # crea el entorno virtual 
```

* Activar entorno virtual (siempre).

``` bash
    source yolo_inference/bin/activate # activa el entorno virtual
    deactivate # desactiva el entorno virtual
    rm -r env # eliminar el entorno virtual
```

* Instalar librerias en el entorno virtual creado (solo la primera vez, a menos que se actualice).

``` bash
    pip3 install -r requirements.txt 
```

## Importando librerias

In [28]:
import sys

sys.path.append('../../script')

import ibmCloud as ibm
import gen_txt_inference as txt_inf

ModuleNotFoundError: No module named 'botocore'

## Clonación y construcción de Darknet (por única vez)

In [1]:
# clonar repositorio darknet
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15308, done.
remote: Total 15308 (delta 0), reused 0 (delta 0), pack-reused 15308
Receiving objects: 100% (15308/15308), 13.69 MiB | 8.01 MiB/s, done.
Resolving deltas: 100% (10406/10406), done.


## Compilando Darknet

Modificar el archivo `/darknet/Makefile`. Una opción es abrirlo directamente o usando las siguientes instrucciones.

In [3]:
%cd darknet

/home/max/Dropbox/Git/proyectos/UALI/ml_uali/inference/darknet


In [4]:
# cambie el makefile para tener GPU y OPENCV habilitados (para pruebas locales)
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=1/GPU=0/' Makefile
!sed -i 's/CUDNN=1/CUDNN=0/' Makefile
!sed -i 's/CUDNN_HALF=1/CUDNN_HALF=0/' Makefile

Modificar las siguientes lineas para redireccionar el path de CUDA

```
GPU=1
CUDNN=1

ifeq ($(GPU), 1)
COMMON+= -DGPU -I/usr/local/cuda-11.4/include/
CFLAGS+= -DGPU
ifeq ($(OS),Darwin) #MAC
LDFLAGS+= -L/usr/local/cuda-11.4/lib -lcuda -lcudart -lcublas -lcurand
else
LDFLAGS+= -L/usr/local/cuda-11.4/lib64 -lcuda -lcudart -lcublas -lcurand
endif
endif

ifeq ($(CUDNN), 1)
COMMON+= -DCUDNN
ifeq ($(OS),Darwin) #MAC
CFLAGS+= -DCUDNN -I/usr/local/cuda-11.4/include
LDFLAGS+= -L/usr/local/cuda-11.4/lib -lcudnn
else
CFLAGS+= -DCUDNN -I/usr/local/cudnn/include
LDFLAGS+= -L/usr/local/cudnn/lib64 -lcudnn
endif
endif
```

> **Nota:** Tambien se puede usar el archivo `Makefile_mod` para reemplazar (recordar modificar el nombre)

In [5]:
# make darknet (crea darknet para que luego pueda usar el archivo ejecutable darknet para ejecutar o entrenar detectores de objetos)
!make -h

Usage: make [options] [target] ...
Options:
  -b, -m                      Ignored for compatibility.
  -B, --always-make           Unconditionally make all targets.
  -C DIRECTORY, --directory=DIRECTORY
                              Change to DIRECTORY before doing anything.
  -d                          Print lots of debugging information.
  --debug[=FLAGS]             Print various types of debugging information.
  -e, --environment-overrides
                              Environment variables override makefiles.
  --eval=STRING               Evaluate STRING as a makefile statement.
  -f FILE, --file=FILE, --makefile=FILE
                              Read FILE as a makefile.
  -h, --help                  Print this message and exit.
  -i, --ignore-errors         Ignore errors from recipes.
  -I DIRECTORY, --include-dir=DIRECTORY
                              Search DIRECTORY for included makefiles.
  -j [N], --jobs[=N]          Allow N jobs at once; infinite jobs with no arg.
  -k, 

In [6]:
!make -d

GNU Make 4.2.1
Built for x86_64-pc-linux-gnu
Copyright (C) 1988-2016 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Reading makefiles...
Reading makefile 'Makefile'...
Updating makefiles....
 Considering target file 'Makefile'.
  Looking for an implicit rule for 'Makefile'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.o'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.c'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.cc'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.C'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.cpp'.
  Trying pattern rule with stem 'Makefile'.
  Trying implicit prerequisite 'Makefile.p'.
  Try

In [7]:
./darknet detector test ./cfg/coco.data ./cfg/yolov4.cfg ./yolov4.weights

SyntaxError: invalid syntax (<ipython-input-7-55b82439c9d7>, line 1)

## Descargando pesos de YOLOv4 (entrenamiento propio)

### Yolov4 - openImage (yolo_v1)

* weights-file (best): --> URL: https://drive.google.com/file/d/1-5eprW8D2Si3gZOqaN4QadHOFhvu6OWT/view?usp=sharing

* weights-file (final) --> URL: https://drive.google.com/file/d/1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H/view?usp=sharing

* data_training (.cfg - .names - .data) --> URL: https://github.com/githubuali/ml_uali/blob/main/training/yolo_v1/data_training.zip?raw=true

### Yolov4 - visDrone (yolo_v2)

* weights-file (best): --> URL: https://drive.google.com/file/d/1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd/view?usp=sharing

* data_training (.cfg - .names - .data) --> URL: https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training.zip?raw=true

### Download

In [11]:
# descargar weights
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd" -O ../yolov4-obj_final.weights && rm -rf /tmp/cookies.txt

--2021-08-31 12:27:46--  https://docs.google.com/uc?export=download&confirm=b0Un&id=1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd
Resolving docs.google.com (docs.google.com)... 2800:3f0:4002:80d::200e, 172.217.173.14
Connecting to docs.google.com (docs.google.com)|2800:3f0:4002:80d::200e|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-bc-docs.googleusercontent.com/docs/securesc/faf0bhb52aohnb283mbiorrlvm7pgbn2/1ohj8td3trd36jp0j5j5qoihfonsrvc6/1630423650000/04394533313268812415/11339251720472366859Z/1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd?e=download [following]
--2021-08-31 12:27:46--  https://doc-10-bc-docs.googleusercontent.com/docs/securesc/faf0bhb52aohnb283mbiorrlvm7pgbn2/1ohj8td3trd36jp0j5j5qoihfonsrvc6/1630423650000/04394533313268812415/11339251720472366859Z/1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd?e=download
Resolving doc-10-bc-docs.googleusercontent.com (doc-10-bc-docs.googleusercontent.com)... 2800:3f0:4002:80c::2001, 172.217.172.225
Connecting 

In [17]:
# descargar comprimido de data_training
!wget -O ../data_training.zip https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training.zip?raw=true

--2021-08-31 12:43:58--  https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training.zip?raw=true
Resolving github.com (github.com)... 20.201.28.151
Connecting to github.com (github.com)|20.201.28.151|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/githubuali/ml_uali/raw/main/training/yolo_v2/data_training.zip [following]
--2021-08-31 12:43:59--  https://github.com/githubuali/ml_uali/raw/main/training/yolo_v2/data_training.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/githubuali/ml_uali/main/training/yolo_v2/data_training.zip [following]
--2021-08-31 12:43:59--  https://raw.githubusercontent.com/githubuali/ml_uali/main/training/yolo_v2/data_training.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.git

### Ordenando archivos

In [18]:
# descomprima los conjuntos de datos 
!unzip ../data_training.zip -d ../data_training

Archive:  ../data_training.zip
   creating: ../data_training/data_training/
   creating: ../data_training/data_training/.ipynb_checkpoints/
  inflating: ../data_training/data_training/.ipynb_checkpoints/obj-checkpoint.data  
  inflating: ../data_training/data_training/.ipynb_checkpoints/obj-checkpoint.names  
  inflating: ../data_training/data_training/.ipynb_checkpoints/yolov4-obj-checkpoint.cfg  
  inflating: ../data_training/data_training/obj.data  
  inflating: ../data_training/data_training/yolov4-obj.cfg  
  inflating: ../data_training/data_training/obj.names  


In [19]:
# copiar obj.name
!cp ../data_training/data_training/obj.names data

In [20]:
# eliminando carpetas
!rm -rf ../data_training.zip

### Ejecutando Darknet y YOLOv4

### Por URL

URL: https://drive.google.com/file/d/1aLTw006n7dTYk_9P8TsY7tN-3xd-txLW/view?usp=sharing

In [ ]:
# descargar comprimido de imagenes de prueba
!wget -O ../image_ypf.zip https://github.com/githubuali/ml_uali/blob/main/data/image_ypf.zip?raw=true

# descomprima los conjuntos de datos 
!unzip ../image_ypf.zip

# eliminando carpetas
!rm -rf ../image_ypf.zip

In [25]:
prefix = 'eventosDetectados_YPF'
download_path = '/home/maximiliano/ml_uali/inference/eventos/' +  prefix

### Por IBM Cloud 

In [12]:
prefix = '28072021_SN_'
download_path = '/home/maximiliano/ml_uali/inference/eventos/' +  prefix

In [13]:
cos = ibm.configInstance()
buckets = ibm.getBuckets(cos)   
keys = ibm.getObjectsNames(cos, buckets[0], prefix) 
ibm.downloadFiles(cos, buckets[0], keys[0::], download_path)

Retrieving list of buckets
Retrieving objects in specified bucket
Starting download of file 28072021_SN_DJI_0001.JPG-----------] 0.0% ...Descarga en proceso
28072021_SN_DJI_0001.JPG donwloaded
Starting download of file 28072021_SN_DJI_0002.JPG-----------] 0.2% ...Descarga en proceso
28072021_SN_DJI_0002.JPG donwloaded
Starting download of file 28072021_SN_DJI_0003.JPG-----------] 0.4% ...Descarga en proceso
28072021_SN_DJI_0003.JPG donwloaded
Starting download of file 28072021_SN_DJI_0004.JPG-----------] 0.6% ...Descarga en proceso
28072021_SN_DJI_0004.JPG donwloaded
Starting download of file 28072021_SN_DJI_0005.JPG-----------] 0.8% ...Descarga en proceso
28072021_SN_DJI_0005.JPG donwloaded
Starting download of file 28072021_SN_DJI_0006.JPG-----------] 1.0% ...Descarga en proceso
28072021_SN_DJI_0006.JPG donwloaded
Starting download of file 28072021_SN_DJI_0007.JPG-----------] 1.2% ...Descarga en proceso
28072021_SN_DJI_0007.JPG donwloaded
Starting download of file 28072021_SN_DJI_000

In [26]:
# generar el .txt con paths
txt_inf.gen_text (download_path+'/')

NameError: name 'txt_inf' is not defined

In [22]:
!ls

3rdparty	DarknetConfig.cmake.in	include		       README.md
backup		darknet_images.py	json_mjpeg_streams.sh  results
build		darknet.py		LICENSE		       scripts
build.ps1	darknet_video.py	Makefile	       src
cfg		data			net_cam_v3.sh	       vcpkg.json
cmake		image_yolov3.sh		net_cam_v4.sh	       video_yolov3.sh
CMakeLists.txt	image_yolov4.sh		obj		       video_yolov4.sh


In [24]:
# Necesitamos configurar nuestro cfg personalizado en modo de prueba (unica vez)
!sed -i 's/batch=64/batch=1/' ../data_training/data_training/yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' ../data_training/data_training/yolov4-obj.cfg

In [25]:
import time
inicio = time.time()

time.strftime('La inferencia se inició: '+'%Y-%m-%d %H:%M:%S', time.localtime())

'La inferencia se inició: 2021-08-03 23:58:51'

In [27]:
#ejecute su detector personalizado con este comando (cargue una imagen en su unidad de Google para probar, la bandera thresh establece la precisión que debe tener la detección para mostrarla)
!./darknet detector test ../data_training/obj.data ../data_training/yolov4-obj.cfg ../yolov4-obj_final.weights -ext_output -dont_show -out result.json < ../eventos/eventosDetectados_YPF/images.txt

time.strftime('La inferencia finalizó: '+'%Y-%m-%d %H:%M:%S', time.localtime())

fin = time.time()
print('El tiempo total de proceso es de: ' + str((fin-inicio)*(1.7*(10**-5)))+ ' minutos.') 

 CUDA-version: 11040 (11040), cuDNN: 8.2.1, GPU count: 1  
 OpenCV version: 4.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   8 conv     64       1 x 1